In [1]:
import langchain
import langchain_community

In [3]:
langchain.__version__

'0.2.14'

In [5]:
langchain_community.__version__

'0.2.12'

In [7]:
import os

In [9]:
from dotenv import load_dotenv

In [6]:
load_dotenv('/home/santhosh/Projects/courses/Pinnacle/.env')

True

In [11]:
load_dotenv('D:/projects/env')

True

In [13]:
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

# Data

In [16]:
from langchain_community.document_loaders import UnstructuredWordDocumentLoader, UnstructuredPDFLoader

In [17]:
from langchain_community.document_loaders import PyMuPDFLoader

In [20]:
from langchain_text_splitters import CharacterTextSplitter

https://info.email.online.hbs.edu/strategy-formulation-ebook

In [23]:
loader = PyMuPDFLoader(file_path='how-to-formulate-successful-business-strategy.pdf', extract_images=False)

In [25]:
data = loader.load()

In [27]:
len(data)

23

In [29]:
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=500,
    chunk_overlap=0,
    is_separator_regex=False,
)

In [31]:
texts = text_splitter.split_documents(data)

In [33]:
len(texts)

69

In [35]:
texts[0]

Document(metadata={'source': 'how-to-formulate-successful-business-strategy.pdf', 'file_path': 'how-to-formulate-successful-business-strategy.pdf', 'page': 0, 'total_pages': 23, 'format': 'PDF 1.7', 'title': 'How to Formulate a Successful Business Strategy', 'author': 'Harvard Business School Online', 'subject': '', 'keywords': '', 'creator': 'Adobe InDesign 17.4 (Macintosh)', 'producer': 'Adobe PDF Library 16.0.7', 'creationDate': "D:20221018124609-04'00'", 'modDate': "D:20221018124635-04'00'", 'trapped': ''}, page_content='How to Formulate \na\xa0Successful \nBusiness\xa0Strategy')

# Embeddings

In [38]:
from langchain_openai import OpenAIEmbeddings

In [39]:
embedding_model = OpenAIEmbeddings(model='text-embedding-3-small', show_progress_bar=True)

In [41]:
from langchain_chroma import Chroma

In [68]:
db = Chroma.from_documents(documents=texts, embedding=embedding_model, persist_directory='./strategy_db',
                       collection_name='business-strategy', collection_metadata={"hnsw:space": "cosine"})

  0%|          | 0/1 [00:00<?, ?it/s]

In [44]:
db = Chroma(persist_directory="./strategy_db/", 
            collection_name='business-strategy', embedding_function=embedding_model)

In [46]:
retriever = db.as_retriever(search_type="mmr", search_kwargs={"k": 3})

In [48]:
docs = retriever.invoke("wat role do employees play?")

  0%|          | 0/1 [00:00<?, ?it/s]

In [52]:
for i, doc in enumerate(docs):
    # print(doc.metadata)
    print(i, doc.page_content, end='\n\n')

0 aware of your company’s goals, how and why you chose those goals, and what they can 
do to help reach them, you can create an increased sense of responsibility throughout 
your organization.
This can also have trickle-down effects. For instance, if a manager isn’t clear on your 
organization’s strategy or the reasoning used to craft it, they could make decisions that 
counteract it. With one vision to unite around, everyone at your organization can act 
with a broader strategy in mind.

1 How can my business create value for employees? 
And how can my business create value by 
collaborating with suppliers? Think of a company’s 
strategy as an answer to these three questions.”
HBS Professor Felix Oberholzer-Gee 
in Business Strategy

2 generates for its 
shareholders. 
PEOPLE
refers to a company’s 
commitment to making 
a societal impact in 
communities locally and 
abroad. 
PLANET
refers to the impact 
an organization 
makes on the 
environment.



In [54]:
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import chain

In [56]:
@chain
def retriever(query: str) -> List[Document]:
    docs, scores = zip(*db.similarity_search_with_score(query, k=3))
    for doc, score in zip(docs, scores):
        doc.metadata["score"] = score

    return docs

In [58]:
docs = retriever.invoke("wat role do employees play?")

  0%|          | 0/1 [00:00<?, ?it/s]

In [60]:
for i, doc in enumerate(docs):
    print(doc.metadata['score'])
    print(i, doc.page_content, end='\n\n')

0.5587170512512736
0 aware of your company’s goals, how and why you chose those goals, and what they can 
do to help reach them, you can create an increased sense of responsibility throughout 
your organization.
This can also have trickle-down effects. For instance, if a manager isn’t clear on your 
organization’s strategy or the reasoning used to craft it, they could make decisions that 
counteract it. With one vision to unite around, everyone at your organization can act 
with a broader strategy in mind.

0.5931993625165801
1 How can my business create value for employees? 
And how can my business create value by 
collaborating with suppliers? Think of a company’s 
strategy as an answer to these three questions.”
HBS Professor Felix Oberholzer-Gee 
in Business Strategy

0.6429791127071118
2 execution requires organization-wide effort.
Crafting a strategy is crucial, but it can’t be successful unless 
it’s effectively and artfully communicated to all employees 
so they feel empowered 